In [ ]:
# Exploratory Data Analysis: Soybeans vs Brazilian Real

This notebook performs comprehensive EDA on the relationship between soybean futures (ZS=F) and USD/BRL exchange rates.


In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from statsmodels.tsa.stattools import adfuller, coint
from scipy import stats
import warnings
warnings.filterwarnings('ignore')

plt.style.use('seaborn-v0_8')
sns.set_palette("husl")
plt.rcParams['figure.figsize'] = (12, 8)


In [ ]:
## 1. Data Loading and Basic Statistics


In [ ]:
# Load the data
df = pd.read_csv("../data/raw_merged.csv", index_col=0, parse_dates=True)
print(f"Data shape: {df.shape}")
print(f"Date range: {df.index.min()} to {df.index.max()}")
print(f"Missing values: {df.isnull().sum().to_dict()}")

df.head()


In [ ]:
# Basic descriptive statistics
print("Descriptive Statistics:")
print(df.describe())


In [ ]:
## 2. Scatter Plot and Correlation Analysis


In [ ]:
# Scatter plot with regression line
plt.figure(figsize=(10, 8))
sns.scatterplot(data=df, x='soy_usd', y='usdbrl', alpha=0.6, s=20)
sns.regplot(data=df, x='soy_usd', y='usdbrl', scatter=False, color='red', linewidth=2)

# Calculate correlation
correlation = df['soy_usd'].corr(df['usdbrl'])
plt.title(f'Soybean Price vs USD/BRL Exchange Rate\nPearson ρ = {correlation:.4f}', 
          fontsize=14, fontweight='bold')
plt.xlabel('Soybean Price (USD/bu)')
plt.ylabel('USD/BRL Exchange Rate')
plt.grid(True, alpha=0.3)
plt.show()

print(f"Pearson correlation coefficient: {correlation:.4f}")


In [ ]:
## 3. ADF Stationarity Tests


In [ ]:
def adf_test(series, name):
    """Perform Augmented Dickey-Fuller test"""
    result = adfuller(series.dropna())
    print(f"\nADF Test Results for {name}:")
    print(f"ADF Statistic: {result[0]:.6f}")
    print(f"p-value: {result[1]:.6f}")
    
    if result[1] <= 0.05:
        print(f"✓ {name} is stationary (reject null hypothesis)")
    else:
        print(f"✗ {name} is non-stationary (fail to reject null hypothesis)")

# Test levels
adf_test(df['soy_usd'], 'Soybean Price (Levels)')
adf_test(df['usdbrl'], 'USD/BRL (Levels)')

# Calculate log returns
df['soy_returns'] = np.log(df['soy_usd']).diff()
df['brl_returns'] = np.log(df['usdbrl']).diff()

# Test log returns
adf_test(df['soy_returns'], 'Soybean Log Returns')
adf_test(df['brl_returns'], 'USD/BRL Log Returns')


In [ ]:
## 4. Engle-Granger Cointegration Test


In [ ]:
# Engle-Granger cointegration test
clean_data = df[['soy_usd', 'usdbrl']].dropna()
coint_result = coint(clean_data['usdbrl'], clean_data['soy_usd'])

print("Engle-Granger Cointegration Test:")
print(f"Cointegration Statistic: {coint_result[0]:.6f}")
print(f"p-value: {coint_result[1]:.6f}")

if coint_result[1] <= 0.05:
    print("✓ Series are cointegrated (reject null hypothesis)")
else:
    print("✗ Series are not cointegrated (fail to reject null hypothesis)")


In [ ]:
## 5. Rolling 250-Day Beta Analysis


In [ ]:
from sklearn.linear_model import LinearRegression

def calculate_rolling_beta(df, window=250):
    """Calculate rolling beta using 250-day window"""
    betas = []
    
    for i in range(window, len(df)):
        window_data = df.iloc[i-window:i]
        clean_window = window_data[['soy_usd', 'usdbrl']].dropna()
        
        if len(clean_window) < window * 0.8:
            betas.append(np.nan)
            continue
        
        X = clean_window['soy_usd'].values.reshape(-1, 1)
        y = clean_window['usdbrl'].values
        
        reg = LinearRegression().fit(X, y)
        beta = reg.coef_[0]
        betas.append(beta)
    
    return pd.Series(betas, index=df.index[window:])

# Calculate rolling beta
rolling_beta = calculate_rolling_beta(df)

print(f"Beta statistics:")
print(f"Mean: {rolling_beta.mean():.4f}")
print(f"Std: {rolling_beta.std():.4f}")
print(f"Min: {rolling_beta.min():.4f}")
print(f"Max: {rolling_beta.max():.4f}")


In [ ]:
# Plot rolling beta
plt.figure(figsize=(15, 8))
plt.plot(rolling_beta.index, rolling_beta.values, linewidth=1, alpha=0.8, color='darkblue')
plt.axhline(y=rolling_beta.mean(), color='red', linestyle='--', alpha=0.7, 
           label=f'Mean β = {rolling_beta.mean():.4f}')
plt.fill_between(rolling_beta.index, 
                rolling_beta.mean() - rolling_beta.std(),
                rolling_beta.mean() + rolling_beta.std(),
                alpha=0.2, color='red', label='±1σ Band')
plt.title('Rolling 250-Day Beta: USD/BRL vs Soybean Prices', fontsize=14, fontweight='bold')
plt.ylabel('Beta Coefficient')
plt.xlabel('Date')
plt.legend()
plt.grid(True, alpha=0.3)
plt.show()


In [ ]:
## 6. Z-Score Regime Plot (2010-2023)


In [ ]:
def calculate_rolling_zscore(df, window=250):
    """Calculate rolling regression z-scores"""
    z_scores = []
    
    for i in range(window, len(df)):
        window_data = df.iloc[i-window:i]
        clean_window = window_data[['soy_usd', 'usdbrl']].dropna()
        
        if len(clean_window) < window * 0.8:
            z_scores.append(np.nan)
            continue
        
        X = clean_window['soy_usd'].values.reshape(-1, 1)
        y = clean_window['usdbrl'].values
        
        reg = LinearRegression().fit(X, y)
        
        # Current observation
        current_soy = df.iloc[i]['soy_usd']
        current_brl = df.iloc[i]['usdbrl']
        
        if pd.isna(current_soy) or pd.isna(current_brl):
            z_scores.append(np.nan)
            continue
        
        # Calculate z-score
        predicted = reg.predict([[current_soy]])[0]
        residual = current_brl - predicted
        
        window_predictions = reg.predict(X)
        window_residuals = y - window_predictions
        residual_std = np.std(window_residuals)
        
        z_score = residual / residual_std if residual_std > 0 else 0
        z_scores.append(z_score)
    
    return pd.Series(z_scores, index=df.index[window:])

# Calculate z-scores
z_scores = calculate_rolling_zscore(df)

print(f"Z-score statistics:")
print(f"Mean: {z_scores.mean():.4f}")
print(f"Std: {z_scores.std():.4f}")
print(f"Values > 2σ: {(abs(z_scores) > 2).sum()} ({(abs(z_scores) > 2).mean()*100:.1f}%)")
print(f"Values > 1.5σ: {(abs(z_scores) > 1.5).sum()} ({(abs(z_scores) > 1.5).mean()*100:.1f}%)")


In [ ]:
# Plot z-score regime for 2010-2023
regime_data = z_scores[:'2023-12-31']

fig, (ax1, ax2) = plt.subplots(2, 1, figsize=(15, 10))

# Z-score time series
ax1.plot(regime_data.index, regime_data.values, linewidth=1, alpha=0.8, color='navy')
ax1.axhline(y=2, color='red', linestyle='--', alpha=0.7, label='±2σ Trading Bands')
ax1.axhline(y=-2, color='red', linestyle='--', alpha=0.7)
ax1.axhline(y=1.5, color='orange', linestyle='--', alpha=0.7, label='±1.5σ Fallback Bands')
ax1.axhline(y=-1.5, color='orange', linestyle='--', alpha=0.7)
ax1.axhline(y=0, color='black', linestyle='-', alpha=0.3)
ax1.fill_between(regime_data.index, -2, 2, alpha=0.1, color='red')
ax1.set_title('Z-Score Regime Analysis (2010-2023)', fontsize=14, fontweight='bold')
ax1.set_ylabel('Z-Score')
ax1.legend()
ax1.grid(True, alpha=0.3)

# Z-score histogram
ax2.hist(regime_data.dropna(), bins=50, alpha=0.7, density=True, color='skyblue', edgecolor='black')
ax2.axvline(x=2, color='red', linestyle='--', linewidth=2, label='2σ Threshold')
ax2.axvline(x=-2, color='red', linestyle='--', linewidth=2)
ax2.axvline(x=1.5, color='orange', linestyle='--', linewidth=2, label='1.5σ Fallback')
ax2.axvline(x=-1.5, color='orange', linestyle='--', linewidth=2)
ax2.set_title('Z-Score Distribution', fontsize=12)
ax2.set_xlabel('Z-Score')
ax2.set_ylabel('Density')
ax2.legend()
ax2.grid(True, alpha=0.3)

plt.tight_layout()
plt.show()
